In [ ]:
import datetime as dt
import dill
import itertools
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import ot
from p_tqdm import p_map
import pandas as pd
import seaborn as sns
import time
from tqdm.notebook import tqdm

In [ ]:
dill.settings['recurse'] = True

In [ ]:
np.random.seed(315)

In [ ]:
output_dir = "C:\\Users\\oconn\\Dropbox\\Research\\EstimationOfOJ\\Simulation"

# Markov and Hidden Markov Chain Logic

In [ ]:
class MarkovChain:
    def __init__(self, transition_matrix, states):
        self.transition_matrix = transition_matrix
        self.stationary_distribution = self.get_stationary_distribution(transition_matrix)
        self.states = states
        
    @staticmethod
    def get_stationary_distribution(transition_matrix):
        eig_vals, eig_vecs = np.linalg.eig(transition_matrix)
        stationary_distribution = np.real(eig_vecs[:, np.isclose(eig_vals, 1)][:, 0])
        stationary_distribution /= np.sum(stationary_distribution)
        return stationary_distribution
        
    def sample(self, n):
        sampled_idxs = []
        
        # Sample from stationary distribution first
        sampled_idxs.append(np.random.choice(len(self.states), p=self.stationary_distribution))
        
        # Sample remaining from transition matrix
        for i in range(1, n):
            sampled_idxs.append(np.random.choice(
                len(self.states), 
                p=self.transition_matrix[sampled_idxs[-1],:]
            ))

        return [self.states[i] for i in sampled_idxs]
    
    
class HiddenMarkovChain:
    def __init__(self, transition_matrix, emission_matrix, states):
        self.markov_chain = MarkovChain(transition_matrix, list(np.arange(transition_matrix.shape[0])))
        self.emission_matrix = emission_matrix
        self.states = states
                
    def sample(self, n):
        latent_samples = self.markov_chain.sample(n)
        
        sampled_idxs = []
        for i in range(n):
            sampled_idxs.append(np.random.choice(
                len(self.states),
                p=self.emission_matrix[latent_samples[i],:]
            ))
        
        return [self.states[i] for i in sampled_idxs]

In [ ]:
def gen_mc(n_states):
    transition_matrix = np.abs(np.random.normal(size=(n_states, n_states)))
    transition_matrix = transition_matrix / np.sum(transition_matrix, axis=1)[:,None]

    return MarkovChain(
        transition_matrix=transition_matrix,
        states=np.linspace(1, n_states, num=n_states, dtype=np.int32)
    )


def gen_hmm(n_hidden_states, n_states):
    transition_matrix = np.abs(np.random.normal(size=(n_hidden_states, n_hidden_states)))
    transition_matrix = transition_matrix / np.sum(transition_matrix, axis=1)[:,None]
    
    emission_matrix = np.abs(np.random.normal(size=(n_hidden_states, n_states)))
    emission_matrix = emission_matrix / np.sum(emission_matrix, axis=1)[:,None]
    
    return HiddenMarkovChain(
        transition_matrix=transition_matrix,
        emission_matrix=emission_matrix,
        states=np.linspace(1, n_states, num=n_states, dtype=np.int32)
    )

# Optimal Joining Estimation Logic

In [ ]:
def get_kblock_measure(x_arr, k):
    n = len(x_arr)
    prob_dict = {}
    for i in range(n-k+1):
        block = tuple(x_arr[i:(i+k)])
        if block in prob_dict:
            prob_dict[block] += 1/n
        else:
            prob_dict[block] = 1/n
    return prob_dict

def get_cost_dict(mu, nu, c):
    cost_dict = {}
    for block_x in mu:
        for block_y in nu:
            cost_dict[block_x, block_y] = np.mean(np.array([c(x, y) for x, y in zip(block_x, block_y)]))
    return cost_dict

def melt_coupling(coupling_arr, x_block_arr, y_block_arr):
    support_points = zip(np.where(coupling_arr > 0)[0], np.where(coupling_arr > 0)[1])
    _list = []
    for i, j in support_points:
        _list.append(pd.DataFrame({
            "x_block": [x_block_arr[i]],
            "y_block": [y_block_arr[j]],
            "probability": [coupling_arr[i, j]]
        }))
        
    return pd.concat(_list).sort_values("probability", ascending=False).reset_index(drop=True)

def display_coupling(melted_coupling, n_points=10):
    display(
        melted_coupling
        .head(n_points)
        .style
        .background_gradient(subset="probability")
        .set_precision(2)
    )

In [ ]:
def estimate_oj(x_arr, y_arr, c, k, reg=0, mode="cost"):
    # Args:
    # - x_arr, y_arr: numpy arrays containing observed sequences
    # - c: cost function
    # - k: block size
    # - reg: regularization coefficient
    # - mode: "cost", "cost-time", or "all"
    
    # Construct k-block measures
    mu_dict = get_kblock_measure(x_arr, k)
    nu_dict = get_kblock_measure(y_arr, k)

    # Construct cost matrix
    cost_dict = get_cost_dict(mu_dict, nu_dict, c)
    
    # Convert to numpy arrays
    x_block_arr = list(mu_dict.keys())
    mu = np.array([mu_dict[x_block] for x_block in x_block_arr])
    y_block_arr = list(nu_dict.keys())
    nu = np.array([nu_dict[y_block] for y_block in y_block_arr])
    cost_mat = np.array([[cost_dict[x_block, y_block] for y_block in y_block_arr] for x_block in x_block_arr])
    
    # Solve OT problem
    start_time = time.time()
    if reg == 0:
        coupling_arr = ot.emd(mu, nu, cost_mat)
    else:
        coupling_arr = ot.sinkhorn(mu, nu, cost_mat, reg/k, method="sinkhorn", stopThr=1e-4)
    end_time = time.time()
    total_time = end_time - start_time
    
    estimated_cost = np.sum(coupling_arr*cost_mat)
    if mode == "cost":
        return estimated_cost
    elif mode == "cost-time":
        return estimated_cost, total_time, start_time, end_time
    elif mode == "all":
        return (x_block_arr, mu), (y_block_arr, nu), coupling_arr, np.sum(coupling_arr*cost_mat)

# Experiment 1: Estimated OJ Cost as a Function of n and k

In [ ]:
def compute_cost_for_grid(x_arr, y_arr, step_size=1):
    c = lambda x, y: int(x != y)
    n_grid = np.arange(10, len(x_arr), step=step_size, dtype=np.int32)
    
    _list = []
    for n in tqdm(n_grid):
        k_grid = np.arange(1, n, step=step_size, dtype=np.int32)
        for k in k_grid:
            oj_cost, total_time = estimate_oj(x_arr[:n], y_arr[:n], c, k, reg=0, mode="cost-time")
            _list.append(pd.DataFrame({
                "n": [n],
                "k": [k],
                "estimated_cost": [oj_cost],
                "time": [total_time]
            }))

    return pd.concat(_list, ignore_index=True).pivot("k", "n", "estimated_cost")

In [ ]:
def compute_cost_for_logn(x_arr, y_arr, step_size=1, reg=0):
    c = lambda x, y: int(x != y)
    n_grid = np.arange(50, len(x_arr)+1, step=step_size, dtype=np.int32)
    
    _list = []
    for n in n_grid:
        k = max(int(0.25*np.log2(n)), 1)
        oj_cost, _, _, _ = estimate_oj(x_arr[:n], y_arr[:n], c, k, reg=reg, mode="cost-time")
        _list.append(pd.DataFrame({
            "n": [n],
            "k": [k],
            "estimated_cost": [oj_cost],
        }))

    return pd.concat(_list, ignore_index=True)

In [ ]:
def run_cost_convergence_experiment(sampling_func1, sampling_func2, n_iter, max_n, reg, exp_dir, run_name):        
    def iter_func(i):
        x_arr = sampling_func1(max_n)
        y_arr = sampling_func2(max_n)

        # Without regularization
        df = compute_cost_for_logn(x_arr, y_arr, step_size=50)
        df["reg"] = 0
        
        # With regularization
        df_reg = compute_cost_for_logn(x_arr, y_arr, step_size=50, reg=reg)
        df_reg["reg"] = reg
        
        return pd.concat([df, df_reg], ignore_index=True)
    
    list_df = p_map(iter_func, list(range(n_iter)))
    
    results_df = (
        pd.concat(list_df)
        .groupby(["n", "reg"])
        .agg(
            mean=("estimated_cost", np.mean),
            sd=("estimated_cost", np.std),
            lower_quantile=("estimated_cost", lambda x: np.quantile(x, q=0.025)),
            upper_quantile=("estimated_cost", lambda x: np.quantile(x, q=0.975))
        )
        .reset_index()
    )
    results_df["lower_sd"] = np.maximum(results_df["mean"] - 2*results_df["sd"], 0)
    results_df["upper_sd"] = np.minimum(results_df["mean"] + 2*results_df["sd"], 1)
        
    file_name = run_name + "_results_df.pkl"
    results_df.to_pickle(os.path.join(exp_dir, file_name))
        
    return results_df


def plot_cost_convergence_results(results_df, name):
    for reg, _df in results_df.groupby("reg"):
        plt.plot(_df["n"], _df["mean"], label="reg={}".format(reg))

    plt.legend()

    for reg, _df in results_df.groupby("reg"):
        plt.fill_between(_df["n"], _df["lower_sd"], _df["upper_sd"], alpha=0.3, label="reg={}".format(reg))

    plt.xlabel("n")
    plt.ylabel("Estimated Cost")
    plt.title(name)
    plt.grid()
    plt.show()

In [ ]:
# Setup experiment directory
exp_id = ''.join(c for c in str(dt.datetime.now()) if c.isdigit())
exp_dir = os.path.join(output_dir, "cost_convergence", exp_id)
os.makedirs(exp_dir, exist_ok=True)
print(exp_dir)


# Experiment parameters
max_n = 2500
n_iter = 50
reg = 5e-1


# Identical Markov Chains
print("identical Markov chains")
mc = gen_mc(n_states=2)
results_df = run_cost_convergence_experiment(
    sampling_func1=mc.sample,
    sampling_func2=mc.sample,
    n_iter=n_iter,
    max_n=max_n,
    reg=reg,
    exp_dir=exp_dir,
    run_name="identical_markov_chains"
)
plot_cost_convergence_results(results_df, name="Identical Markov Chains")


# Identical HMMs
print("identical HMMs")
hmm = gen_hmm(n_hidden_states=5, n_states=2)
results_df = run_cost_convergence_experiment(
    sampling_func1=hmm.sample,
    sampling_func2=hmm.sample,
    n_iter=n_iter,
    max_n=max_n,
    reg=reg,
    exp_dir=exp_dir,
    run_name="identical_hmms"
)
plot_cost_convergence_results(results_df, name="Identical HMMs")


# MC vs MC
print("MC vs MC")
mc1 = gen_mc(n_states=2)
mc2 = gen_mc(n_states=2)
results_df = run_cost_convergence_experiment(
    sampling_func1=mc1.sample,
    sampling_func2=mc2.sample,
    n_iter=n_iter,
    max_n=max_n,
    reg=reg,
    exp_dir=exp_dir,
    run_name="mc_vs_mc"
)
plot_cost_convergence_results(results_df, name="Markov Chain vs Markov Chain")


# HMM vs HMM
print("HMM vs HMM")
hmm1 = gen_hmm(n_hidden_states=5, n_states=2)
hmm2 = gen_hmm(n_hidden_states=5, n_states=2)
results_df = run_cost_convergence_experiment(
    sampling_func1=hmm1.sample,
    sampling_func2=hmm2.sample,
    n_iter=n_iter,
    max_n=max_n,
    reg=reg,
    exp_dir=exp_dir,
    run_name="hmm_vs_hmm"
)
plot_cost_convergence_results(results_df, name="HMM vs HMM")


# Experiment 2: Runtime for Entropic OJ Estimate

In [ ]:
n_iter = 50
n_grid = [1000, 1500, 2000, 2500]
reg_grid = [1e-1, 1.5e-1, 2e-1]

c = lambda x, y: int(x != y)

n_hidden_states = 100
n_states = 10

In [ ]:
_list = []
for n in n_grid:
    print("n={}".format(n))
    k = max(int(0.5*np.log2(n)), 1)
    for _ in tqdm(range(n_iter)):
        hmm1 = gen_hmm(n_hidden_states, n_states)
        hmm2 = gen_hmm(n_hidden_states, n_states)
        
        x_arr = hmm1.sample(n)
        y_arr = hmm2.sample(n)

        for reg in reg_grid:
            est_cost, total_time, start_time, end_time = estimate_oj(x_arr, y_arr, c, k, reg=0, mode="cost-time")
            est_cost_reg, total_time_reg, start_time_reg, end_time_reg = estimate_oj(x_arr, y_arr, c, k, reg=reg, mode="cost-time")
            _list.append(pd.DataFrame({
                "n": [n],
                "k": [k],
                "est_cost": [est_cost],
                "est_cost_reg": [est_cost_reg],
                "total_time": [total_time],
                "total_time_reg": [total_time_reg],
                "start_time": [start_time],
                "end_time": [end_time],
                "start_time_reg": [start_time_reg],
                "end_time_reg": [end_time_reg],
                "reg": [reg]
            }))

results_df = pd.concat(_list, ignore_index=True)

results_df["time_diff"] = results_df["total_time"] - results_df["total_time_reg"]
results_df["time_diff_pct"] = 100 * results_df["time_diff"] / results_df["total_time"]
results_df["cost_diff"] = (results_df["est_cost_reg"] - results_df["est_cost"]).abs()
results_df["cost_diff_pct"] = 100 * results_df["cost_diff"] / results_df["est_cost"]

exp_id = ''.join(c for c in str(dt.datetime.now()) if c.isdigit())
exp_dir = os.path.join(output_dir, exp_id)
os.makedirs(exp_dir, exist_ok=True)
results_df.to_pickle(os.path.join(exp_dir, "time_df.pkl"))

display(results_df)